# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [59]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [60]:
csv_path = "city_data_2020-09-28.csv"
city_weather_df = pd.read_csv(csv_path)
city_weather_df.head()


,City,Lat,Long,Max Temp (C),Humidity,Cloudiness,Windspeed,Country Code,Date
0,Dengzhou,37.81,120.75,18.93,74,100,3.52,CN,1601310100
1,Sitka,57.05,-135.33,12.00,76,90,3.60,US,1601310101
2,Zapolyarnyy,69.42,30.81,9.00,87,75,8.20,RU,1601310101
3,Georgetown,-7.93,-14.42,23.63,77,100,7.14,SH,1601310101
4,Rikitea,-23.12,-134.97,23.53,81,3,3.81,PF,1601310102


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [61]:
gmaps.configure(api_key=gkey)

In [63]:
fig = gmaps.figure()

locations = city_weather_df[["Lat","Long"]].astype(float)
humidity = city_weather_df["Humidity"]
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [64]:
#Filtering for cities with crisp, cool weather
ideal_cities_df = city_weather_df.loc[
    (city_weather_df["Max Temp (C)"] < 23) &
    (city_weather_df["Max Temp (C)"] > 15) &
    (city_weather_df["Windspeed"]<15) &
    (city_weather_df["Cloudiness"]<10)
]

ideal_cities_df = ideal_cities_df.dropna(how="any")
ideal_cities_df

,City,Lat,Long,Max Temp (C),Humidity,Cloudiness,Windspeed,Country Code,Date
61,Carnarvon,-24.87,113.63,19.00,77,0,9.30,AU,1601310121
108,Farafangana,-22.82,47.83,22.39,81,0,5.40,MG,1601310138
156,Port Alfred,-33.59,26.89,15.50,53,0,2.56,ZA,1601310152
163,Manakara,-22.13,48.02,22.06,83,0,4.71,MG,1601310155
171,East London,-33.02,27.91,17.00,59,7,5.70,ZA,1601310133
207,Saldanha,-33.01,17.94,16.00,59,0,4.60,ZA,1601310169
240,Guadalupe y Calvo,26.10,-106.97,21.51,35,0,3.43,MX,1601310180
317,Isangel,-19.55,169.27,22.18,84,6,2.89,VU,1601310203
322,Fallon,39.47,-118.78,15.56,32,1,3.55,US,1601310205
327,Eureka,40.80,-124.16,18.89,90,0,2.28,US,1601310206


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
#Creating df with Hotel Name Column
hotel_df = pd.DataFrame({
    "Hotel Name":[],
    "City":[],
    "Country":[],
    "Lat":[],
    "Lng":[]
})

#Setting parameters for API calls
params = {
    "keyword":"hotel",
    "radius":5000,
    "type":"lodging",
    "key":gkey,
    }

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

hotel_df

,Hotel Name,City,Country,Lat,Lng


In [51]:
#Make API calls while looing through ideal_cities_df

for index, row in ideal_cities_df.iterrows():
    lat = row["Lat"]
    long = row["Long"]
    city = row["City"]
    country= row["Country Code"]
    target_coordinates = f"{lat},{long}"
    params["location"] = target_coordinates
    
    json_response = requests.get(base_url,params=params).json()
    
    try:
        hotel_df.loc[index,"Hotel Name"] = json_response["results"][0]["name"]
        hotel_df.loc[index,"Lat"] = json_response["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index,"Lng"] = json_response["results"][0]["geometry"]["location"]["lng"]
        hotel_df.loc[index,"City"] = city
        hotel_df.loc[index,"Country"] = country
        print(f"Storing info for {city}")
    except (KeyError, IndexError):
        print("Something didn't work right...moving on...")
    

Storing info for Carnarvon
Storing info for Farafangana
Storing info for Port Alfred
Storing info for Manakara
Storing info for East London
Storing info for Saldanha
Storing info for Guadalupe y Calvo
Storing info for Isangel
Storing info for Fallon
Storing info for Eureka
Storing info for North Platte
Storing info for Pacific Grove
Storing info for Svobody
Storing info for Orlovskiy
Something didn't work right...moving on...
Storing info for Sabāthu
Storing info for San Luis
Storing info for Piranshahr
Storing info for Paysandu
Storing info for Balkanabat


In [54]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [58]:
# Re-create original heatmap
fig = gmaps.figure()
original_locations = city_weather_df[["Lat","Long"]].astype(float)
humidity = city_weather_df["Humidity"]

#Add the heat layer based on humidity
heat_layer = gmaps.heatmap_layer(original_locations, weights=humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius = 1)

#Add a hotel layer using hotel_info
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=5,
    info_box_content=hotel_info
)

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))